In [1]:
import sys
sys.path.append('../')
import pandas as pd
from utils import TextUtil, WordEmbeddingUtil
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import pickle
import tqdm
from config import Config
import numpy as np

C:\Users\tomoka\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [15]:
df = pd.read_excel('../data/train_sub.xlsx')
# df = pd.read_csv('../data/train.csv')

In [3]:
text_util = TextUtil()

In [18]:
# get train label
np.save('../data/train_label.npy', df.target.values)

In [4]:
# text normalization
question_text_np_array = df.question_text.values
for idx, row in enumerate(tqdm.tqdm(question_text_np_array)):
    question_text_np_array[idx] = text_util.text_normalization(row)

100%|██████████████████████████████████████████████████████████████████████████████| 301/301 [00:00<00:00, 5784.80it/s]


In [5]:
# stem words 词性还原
stem_words = []
for row in tqdm.tqdm(question_text_np_array):
    words = text_util.lemmatize_sentence(row)
    stem_words.append(words)

100%|████████████████████████████████████████████████████████████████████████████████| 301/301 [00:04<00:00, 62.52it/s]


In [6]:
# 去除标点与停用词
filter_words = []
for words in tqdm.tqdm(stem_words):
    words = text_util.filter_punctuation(words)
    words = text_util.filter_stop_word(words)
    filter_words.append(words)

100%|█████████████████████████████████████████████████████████████████████████████| 301/301 [00:00<00:00, 18802.38it/s]


预处理完的数据, 为如下的格式
```
[['I', 'pandas'], ['How', 'Quebec', 'nationalist', 'see', 'province', 'nation']]
```

In [7]:
# with open('../data/words.pkl', mode='wb') as f:
#     pickle.dump(file=f, obj=filter_words)

In [8]:
pad_and_cut_words = []
for words in tqdm.tqdm(filter_words):
    if len(words) < Config.SENTENCE_MAX_LEN-2:
        words = [Config.START_CHAR] + words + [Config.PADDING_CHAR] * (Config.SENTENCE_MAX_LEN - 2 - len(words)) + [Config.END_CHAR]
    else:
        words = [Config.START_CHAR] + words[0, Config.SENTENCE_MAX_LEN - 2] + [Config.END_CHAR]
    pad_and_cut_words.append(words)

100%|████████████████████████████████████████████████████████████████████████████| 301/301 [00:00<00:00, 300878.34it/s]


In [9]:
word2vec_util = WordEmbeddingUtil(word2vec_path=Config.WORD2VEC_PATH)

In [16]:
word_embedding_matrix = np.zeros([len(filter_words), Config.SENTENCE_MAX_LEN, Config.EMBEDDING_SIZE], dtype=np.float32)
for i, words in enumerate(tqdm.tqdm(pad_and_cut_words)):
    words_matrix = np.zeros([Config.SENTENCE_MAX_LEN, Config.EMBEDDING_SIZE], dtype=np.float32)
    for idx, word in enumerate(words):
        words_matrix[idx] = word2vec_util.get_word2vec_vec(word)
    word_embedding_matrix[i] = words_matrix

100%|██████████████████████████████████████████████████████████████████████████████| 301/301 [00:00<00:00, 3133.32it/s]


输出一个embedding矩阵，shape如(文本数, 限定最长词数, 词向量长度)  
eg. (4396, 50, 300)

In [13]:
# np.save('../data/word2vec_martix.npy', word_embedding_matrix)